# More SQL

This notebook builds on what we've learned previously (in Week 4 labs) by adding a few additional concepts, as well as some example SQL Interview Questions.  We'll be focusing on queries (`SELECT` statements).

**Outline**:
- Review
- New concepts
- Notes on using SQL on the job
- Example Interview Questions
- Additional References

## Reminder:

We've learned the following operations already:

```sql
SELECT
FROM
JOIN, ON
WHERE
GROUP BY
HAVING
ORDER BY
LIMIT
```

Some notes on each:

**`SELECT`**: The columns in your final result set. Using `SELECT DISTINCT` returns the unique values for each column in your query

**`FROM`**: The table that you'll be getting your data from. Or, if you're going to be using `JOIN`, the first table you'll look at.

**`JOIN`**: Use this to join another table to your existing set of tables, and use `ON` to specifiy a column from each of the two tables you're joining to be used as the key.  There are four different types of `JOIN`: `INNER JOIN`, `OUTER JOIN` (or `FULL JOIN` in some flavors of SQL), `RIGHT JOIN` and `LEFT JOIN`.  This site has a good visualization explaining each: http://www.sql-join.com/sql-join-types/

**`WHERE`**: Use this to extract only those rows that satisfy a certain condition. eg < , > , <> , = , <= , >= ,  `BETWEEN`, `LIKE`, `IN` (more on the last three below).  Multiple conditions can be linked together using `AND`, `OR` or `NOT`.

**`GROUP BY`**: This is needed whenever you're doing an aggregation on one or more columns. NOTE: when using `GROUP BY`, you must group by all columns that are not being aggregated.

**`HAVING`**: Used to return only rows of an **agregated** column that satisfy a certain condition. This is similar to `WHERE`, though `WHERE` doesn't work with aggregated columns.

**`ORDER BY`**: Used to sort the resulting table according to one or more columns.  Use with `ASC` or `DESC`.

**`LIMIT`**: Used to limit the number of rows returned.  Like `.head()` in pandas, though you need to specify a number of rows.


We also learned aggregations like `MIN`, `MAX`, `COUNT`, `AVG` and `SUM`, which can be used in `SELECT`.

## Some new SQL concepts

For the following examples, we'll be using two tables as follows:

**Customers:**

| column | type |
| --|--|
|id (key) | INT |
|customer_name | STR|
|zipcode | INT |
| date_of_birth | DATE |

**Purchases:**

| column | type |
| --|--|
|purchase_id (key) | INT |
|customer_id (foreign key)| INT|
|purchase_total| INT |
|purchase_date | DATE |

### Data Types: Working with Strings

When working with strings, it's safest to convert them to lower case to avoid leakage.  We do this using the `LOWER` keyword.

#### `LIKE`
The `LIKE` operator is used to compare strings to a specified pattern, and is used in the `WHERE` clause. You can also use `NOT LIKE` to find strings that don't follow the given pattern. 

There are two wildcards used with `LIKE`:
- %: matches any string of zero or more characters
- \_: matches any single character

Simple examples:
```sql
'abc' LIKE 'abc'    true
'abc' LIKE 'a%'     true
'abc' LIKE '_b_'    true
'abc' LIKE 'c'      false
```

Example combining `LOWER` and `LIKE`:

Find all customers with names ending with an 'a'.

```sql
SELECT customer_name, zipcode
FROM customers
WHERE LOWER(customer_name) LIKE '%a';
```


### Data Types: Null Values

If a field in a table is optional, it's possible for rows to have a `NULL` value for that field.

To check for `NULL` values in SQL, you can use `IS NULL` or `IS NOT NULL` in the `WHERE` clause.

For example:

Find customers that don't have their zipcode listed:
``` sql
SELECT customer_name AS Name, zipcode
FROM customers
WHERE zipcode IS NULL;
```

Find customers that do have their zipcode listed:
``` sql
SELECT customer_name AS Name, zipcode
FROM customers
WHERE zipcode IS NOT NULL;
```

### Data Types: Dates
Dates are stored in PostgreSQL using the DATE data type. When working with them, DATE types have the appearance `yyyy-mm-dd`.

Similar to using dates in Python, SQL supports operations like +, -, >, <, =, etc with Dates.  These comparisons are done in the `WHERE` clause.

The `EXTRACT` function will return different parts of the date. eg day, month, year.

For example, find customers who were born after 1980:
```sql
SELECT customer_name, date_of_birth,
FROM customers
WHERE EXTRACT(YEAR FROM date_of_birth) > 1980;
```

For more info, see: http://www.postgresqltutorial.com/postgresql-date/



### More Comparisons

#### `IN`

The `IN` operator can be used in the `WHERE` clause to allow searching for multiple values.

For example, find customers who live in the areas with zipcodes 98107, 98108,98109.
```sql
SELECT customer_name, zipcode
FROM customers
WHERE zipcode IN (98107,98108,98109);
```

#### `BETWEEN`

The `BETWEEN` operator allows values to be within a range, and is used in the `WHERE` clause.  Begin and end values are included.

Another way to write the above example is:
```sql
SELECT customer_name, zipcode
FROM customers
WHERE zipcode BETWEEN 98107 AND 98109;
```

### Aliases

For ease of readability and presentation, it's common to use aliases in SQL queries.  They're used both in the `SELECT` and `FROM` clauses.  The `AS` keyword is optional.

eg:  

In the `SELECT` clause:

```sql
SELECT customer_name AS Name, date_of_birth AS Birthday
FROM customers;
```

In both `SELECT` and `FROM` clauses:

```sql
SELECT c.customer_name AS Name, p.purchase_total AS Total
FROM customers as c
INNER JOIN purchases as p on c.id = p.customer_id;
```



### `CASE`

The `CASE` statement is SQL's way of using if/then logic.  It's used in the `SELECT` statement.  You can use it to bucket values in one column according to specified conditions, or to create 'Dummy' columns.

The `CASE` statement contains:
- at least one pair of `WHEN` and `THEN` statements, (like if/then)
- an optional `ELSE` statement

Every `CASE` statement **must** end with an `END` statement.


#### Bucketing values:
For example, label customers born in 1980 or later as `>1980`, and those with an earlier birth year as `<1980` :

```sql
SELECT customer_name, 
    CASE
        WHEN EXTRACT(YEAR FROM date_of_birth) >= 1980 THEN '>1980'
        WHEN EXTRACT(YEAR FROM date_of_birth) < 1980 THEN '<1980'
    END AS birth_year
from customers;
```

This would return something like:

| customer_name | birth_year |
| -- | -- |
| Joe | >1980 |
| Sam | <1980 |
| Jill | <1980 |


#### Creating 'Dummy' Columns

Multiple `CASE` statements in the `SELECT` statement will return multiple columns.


For example, count the number of purchases  that were less than \$10, and the number that were greater than or equal to \$10:

```sql
SELECT 
    COUNT(CASE WHEN purchase_total <  10 THEN purchase_total ELSE NULL END) AS '< $10', 
    COUNT(CASE WHEN purchase_total >= 10 THEN purchase_total ELSE NULL END) AS '>= $10'
FROM purchases;
```

This would return something like:

| <\$10 | >=\$10 |
| -- | -- |
| 654 | 243 |




This page has a really good tutorial on `CASE` statements: https://community.modeanalytics.com/sql/tutorial/sql-case/

### Subqueries or Nested Queries

#### Aggregation
Nested Queries can be used to perform operations in multiple steps. When they're used for this purpose, the nested query goes in the `FROM` clause of the outer query.  When using nested queries for aggregation, think of it as another table.  It's important to give your nested query (eg temporary table) an Alias.

For example:

Find each customer's average purchase total:

```sql
SELECT c.customer_name, AVG(p.total)
FROM 
    (SELECT customer_id, sum(purchase_total) as total
    FROM purchases
    GROUP BY customer_id
    ) AS p
INNER JOIN customers AS c on c.id = p.customer_id
GROUP BY c.customer_name;
```

#### Conditional Logic
Nested Queries can be used inside the outer query's `WHERE`, `JOIN` or `CASE` clause to help define conditions.

For example:

Find the customer (name) with the earliest date of purchase:

```sql
SELECT c.customer_name, p.purchase_date
FROM customers c
INNER JOIN purchases p on c.id = p.customer_id
WHERE p.purchase_date = (SELECT MIN(purchase_date)
                           FROM purchases ps
                           );
```

Here's a good tutorial on Nested Queries: https://community.modeanalytics.com/sql/tutorial/sql-subqueries/

## Notes on using SQL on the job

### Avoid using wildcards

In companies with huge tables, leaving queries unbounded can lead to long runtimes.  For example:

```sql
SELECT *
FROM customers;
```

If the database that you're querying is also being queried by other Analysts (or is being used in Prod), you could impact the performance of the database for others as well. 

Instead, if you're building your query or exploring the database, you can use `LIMIT` to only grab the first few lines. For example:

```sql
SELECT *
FROM customers
LIMIT 5;
```

### Schema

Databases can get complicated! So if you can get your hands on the schema, it'll help you write your queries!

A schema shows you relationships between tables and which elements are keys.  

For example:

![image.png](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS2FHna5nYGP4B8PcOwilaOYcmWxAjKxwSI0dNt-GdvyKcnkgkJew)

### Visualize the result you want

Finally, when starting a new query, it's really helpful to draw the table you want before you start!  That way you can visualize which tables you need to join, which aggregations you need to use, and any edge cases you might need to consider!

## Example Interview Questions

The following are two real examples of SQL interview questions:

### Example Set 1: Customers and Orders

Done in real time.

Answer the questions about the following two tables:

```
customers                              orders

|---------------|---------|           |---------------|---------|
| id            | int     |<-+        | order_id      | int     |
| first_name    | varchar |  |-----+->| customer_id   | int     |
| last_name     | varchar |           | order_amount  | int     |
| zip_code      | varchar |           | order_date    | date    |
|---------------|---------|           |---------------|---------|
          
``` 
 
-- TIME LIMIT 15 MINUTES (GET THROUGH AS MUCH AS YOU CAN)

--1) Get all rows and columns from customers table

--2) Get all rows from customer table and only rows from orders table that match (Dupes are ok)

--3) Get total order amount and total number of orders by customer name where their zip code is 98121 and order date is > 1/1/2015; if a customer has no orders results should show zero

--4) Using answer from #3, get only customers that had more than 2 orders

--5) Using answer from #4 add in a numerical rank for each customer name ordering by the customer with the highest number of orders at the top

--6) Get the total number of orders broken into the following buckets based on total order amount: <\$5, \$5 - \$20, >\$20

--7) Get the second highest order amount per customer, if customer only has one order, order amount should show zero

### Example Set 2: Pizza Orders

Homework assignment.

Let’s say we have a database table of pizza orders.  Each row is one pizza that was ordered and let’s say that it includes these columns and this example data (the table is much larger, of course):

| order_id (INT) | order_date (DATE) | customer_id (INT) | order_item_id (INT) | canceled (CHAR(1)) | size (CHAR(3)) | toppings (VARCHAR(1024)) |
|:--:|:--:|:--:|:--:|:--:|:--:|:--:|
|10001|2016-01-01|9876|1|N|XL|
|10001|2016-01-01|9876|2|N|M|Pepperoni|
|10001|2016-01-01|9876|3|N|L|Pineapple, Ham|
|10003|2016-01-01|4658|1|Y|S|Black Olives, Spicy Peppers|
|10004|2016-01-02|3356|1|N|M|Bacon, Extra Cheese|
|10005|2016-01-03|7653|1|N|XL|Hamburger|
|10005|2016-01-03|7653|2|N|XL|Spicy Pepperoni, Feta Cheese

1.Write a SQL query that results in 1 row per month, with columns of:
- Month
- Total Orders
- Total Pizzas Ordered
- 1 more column for total count of each size that we sold (XL, L, M, S)

2.Write a SQL query that results in 1 row per customer and the following columns:
- Customer_id
- First order date
- Total pizzas ordered in the first order
- Whether any pizzas in the first order were canceled

3.Write a SQL query to see how well our new Spicy Pepperoni topping is selling.

## More references

A slightly more advanced tutorial: https://www.tutorialspoint.com/sql/index.htm

A more interactive tutorial, with different sample databases: https://community.modeanalytics.com/sql/

http://sqlfiddle.com/#!9/9eecb

W3Schools https://www.w3schools.com/sql/default.asp

Join types: http://www.sql-join.com/sql-join-types/